In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import ipywidgets as widgets
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from IPython.display import display
import io

# 🚀 **1. Tạo widget để tải file CSV**
upload_button = widgets.FileUpload(accept='.csv', multiple=False, description="Tải file CSV")
upload_output = widgets.Output()

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'thal']
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'ca']
target_column = 'target'

def on_file_upload(change):
    with upload_output:
        upload_output.clear_output()
        if upload_button.value:
            uploaded_file = list(upload_button.value.values())[0]
            content = uploaded_file['content']
            global df, X, y, X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded
            df = pd.read_csv(io.BytesIO(content))
            print("✅ File CSV đã được tải lên thành công!")
            print("Số hàng trong dữ liệu:", len(df))
            print("Cột trong dữ liệu:", df.columns.tolist())
            print("Dữ liệu mẫu (hàng đầu tiên):")
            print(df.iloc[0])

            X = df[categorical_features + numerical_features]
            y = df[target_column]

            print("5 mẫu có nguy cơ mắc bệnh tim từ dữ liệu gốc:")
            print(df[df[target_column] == 1][categorical_features + numerical_features].head())

            print("Phạm vi giá trị tối đa của các đặc trưng trong các mẫu target=1:")
            for feature in numerical_features:
                print(f"{feature}: max={df[df[target_column] == 1][feature].max()}")

            process_data()

upload_button.observe(on_file_upload, names='value')
display(upload_button, upload_output)

# 🚀 **2. Hàm xử lý dữ liệu và huấn luyện mô hình**
def process_data():
    global X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    model = RandomForestClassifier(n_estimators=500, max_depth=None, class_weight='balanced', random_state=0)
    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_test_transformed)
    print("🎯 Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
    print("📜 Báo cáo phân loại:\n", classification_report(y_test, y_pred))

    print("Dự đoán trên tập kiểm tra (so sánh với thực tế):")
    for i, (pred, actual) in enumerate(zip(y_pred, y_test)):
        print(f"Mẫu {i}: Dự đoán = {pred}, Thực tế = {actual}")

    feature_importance = pd.DataFrame({
        'Feature': preprocessor.get_feature_names_out(),
        'Importance': model.feature_importances_
    })
    print("Độ quan trọng của các đặc trưng:")
    print(feature_importance.sort_values(by='Importance', ascending=False))

    dump(preprocessor, "preprocessor.joblib")
    dump(model, "heart_disease_model.joblib")
    print("✅ Mô hình và bộ tiền xử lý đã được lưu!")
    preprocessor_loaded = load("preprocessor.joblib")
    model_loaded = load("heart_disease_model.joblib")

# 🚀 **3. Tạo giao diện nhập liệu bằng tiếng Việt**
# Tạo style cho description_width
style = {'description_width': '150px'}

# Tạo các widget với layout và style
tuoi = widgets.IntText(description="Tuổi:",
                           style=style,
                           layout=widgets.Layout(width='30%'))
gioi_tinh = widgets.Dropdown(options=[("Nữ", 0), ("Nam", 1)], description="Giới tính:",
                                 style=style,
                                 layout=widgets.Layout(width='30%'))
dau_nguc = widgets.Dropdown(options=[("Đau thắt ngực điển hình", 0), ("Đau thắt ngực không điển hình", 1), ("Đau không phải thắt ngực", 2), ("Không triệu chứng", 3)], description="Loại đau ngực:",
                                style=style,
                                layout=widgets.Layout(width='30%'))
huyet_ap = widgets.FloatText(description="Huyết áp nghỉ (mmHg):",
                                 style=style,
                                 layout=widgets.Layout(width='30%'))
cholesterol = widgets.FloatText(description="Cholesterol (mg/dl):",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))
duong_huyet = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đường huyết cao:",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))
dien_tam_do = widgets.Dropdown(options=[("Bình thường", 0), ("Bất thường ST-T", 1), ("Phì đại thất trái", 2)], description="Điện tâm đồ nghỉ:",
                                  style=style,
                                  layout=widgets.Layout(width='30%'))
nhip_tim = widgets.IntText(description="Nhịp tim tối đa:",
                              style=style,
                              layout=widgets.Layout(width='30%'))
dau_nguc_gang_suc = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đau ngực khi gắng sức:",
                                       style=style,
                                       layout=widgets.Layout(width='30%'))
so_mach = widgets.IntSlider(min=0, max=4, description="Số mạch bị hẹp:",
                               style=style,
                               layout=widgets.Layout(width='30%'))
thalassemia = widgets.Dropdown(options=[("Không xác định", 1), ("Bình thường", 3), ("Khiếm khuyết cố định", 6), ("Khiếm khuyết đảo ngược", 7)], description="Thalassemia:",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))

display(tuoi, gioi_tinh, dau_nguc, huyet_ap, cholesterol, duong_huyet, dien_tam_do, nhip_tim, dau_nguc_gang_suc, so_mach, thalassemia)

# 🔥 Nút bấm dự đoán
button = widgets.Button(description="Dự đoán")
output = widgets.Output()

def du_doan_benh_tim(b):
    with output:
        output.clear_output()
        try:
            input_df = pd.DataFrame({
                'age': [tuoi.value],
                'sex': [gioi_tinh.value],
                'cp': [dau_nguc.value],
                'trestbps': [huyet_ap.value],
                'chol': [cholesterol.value],
                'fbs': [duong_huyet.value],
                'restecg': [dien_tam_do.value],
                'thalach': [nhip_tim.value],
                'exang': [dau_nguc_gang_suc.value],
                'ca': [so_mach.value],
                'thal': [thalassemia.value]
            })
            print("Dữ liệu đầu vào để dự đoán:")
            print(input_df.iloc[0])
            input_data = preprocessor_loaded.transform(input_df)
            prediction = model_loaded.predict(input_data)

            # Thêm ngưỡng thủ công cho trường hợp cực đoan



            print("Kết quả dự đoán:", "Có nguy cơ" if prediction[0] == 1 else "Không nguy cơ")
            if prediction[0] == 1:
                print("🔴 Có nguy cơ mắc bệnh tim!")
            else:
                print("🟢 Không có nguy cơ mắc bệnh tim.")
        except NameError:
            print("⚠️ Vui lòng tải file CSV và huấn luyện mô hình trước!")

button.on_click(du_doan_benh_tim)
display(button, output)

FileUpload(value={}, accept='.csv', description='Tải file CSV')

Output()

IntText(value=0, description='Tuổi:', layout=Layout(width='30%'), style=DescriptionStyle(description_width='15…

Dropdown(description='Giới tính:', layout=Layout(width='30%'), options=(('Nữ', 0), ('Nam', 1)), style=Descript…

Dropdown(description='Loại đau ngực:', layout=Layout(width='30%'), options=(('Đau thắt ngực điển hình', 0), ('…

...

Dropdown(description='Đau ngực khi gắng sức:', layout=Layout(width='30%'), options=(('Không', 0), ('Có', 1)), …

IntSlider(value=0, description='Số mạch bị hẹp:', layout=Layout(width='30%'), max=4, style=SliderStyle(descrip…

Dropdown(description='Thalassemia:', layout=Layout(width='30%'), options=(('Không xác định', 1), ('Bình thường…

Button(description='Dự đoán', style=ButtonStyle())

Output()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd
import numpy as np
import ipywidgets as widgets
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from IPython.display import display
import io

# 🚀 **1. Tạo widget để tải file CSV**
upload_button = widgets.FileUpload(accept='.csv', multiple=False, description="Tải file CSV")
upload_output = widgets.Output()

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'thal']
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'ca']
target_column = 'target'

def on_file_upload(change):
    with upload_output:
        upload_output.clear_output()
        if upload_button.value:
            uploaded_file = list(upload_button.value.values())[0]
            content = uploaded_file['content']
            global df, X, y, X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded
            df = pd.read_csv(io.BytesIO(content))
            print("✅ File CSV đã được tải lên thành công!")
            print("Số hàng trong dữ liệu:", len(df))
            print("Cột trong dữ liệu:", df.columns.tolist())
            print("Dữ liệu mẫu (hàng đầu tiên):")
            print(df.iloc[0])

            X = df[categorical_features + numerical_features]
            y = df[target_column]

            print("5 mẫu có nguy cơ mắc bệnh tim từ dữ liệu gốc:")
            print(df[df[target_column] == 1][categorical_features + numerical_features].head())

            print("Phạm vi giá trị tối đa của các đặc trưng trong các mẫu target=1:")
            for feature in numerical_features:
                print(f"{feature}: max={df[df[target_column] == 1][feature].max()}")

            process_data()

upload_button.observe(on_file_upload, names='value')
display(upload_button, upload_output)

def process_data():
    global X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded

    if X.isnull().any().any():
        print("⚠️ Dữ liệu có giá trị thiếu. Tiến hành xử lý...")
        for col in numerical_features:
            X[col].fillna(X[col].median(), inplace=True)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y)

    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    # Cài đặt lưới siêu tham số để thử nghiệm
    param_dist = {
        'n_estimators': [100, 200, 300, 500, 600],
        'max_depth': [None, 10, 15, 20],
        'min_samples_split': [2, 4, 6],
        'max_features': ['sqrt', 'log2'],
        'class_weight': ['balanced']
    }

    rf = RandomForestClassifier(random_state=42)

    search = RandomizedSearchCV(
        rf, param_distributions=param_dist,
        n_iter=15, cv=5, scoring='accuracy', random_state=42, n_jobs=-1
    )
    search.fit(X_train_transformed, y_train)

    best_model = search.best_estimator_
    y_pred = best_model.predict(X_test_transformed)
    acc = accuracy_score(y_test, y_pred)

    print("🎯 Độ chính xác trên tập kiểm tra:", acc)
    print("📜 Báo cáo phân loại:\n", classification_report(y_test, y_pred))

    if acc > 0.8524590163934426:
        print("✅ Đạt yêu cầu độ chính xác!")

    feature_importance = pd.DataFrame({
        'Feature': preprocessor.get_feature_names_out(),
        'Importance': best_model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    print("📊 Độ quan trọng đặc trưng:")
    print(feature_importance)

    dump(preprocessor, "preprocessor.joblib")
    dump(best_model, "heart_disease_model.joblib")
    preprocessor_loaded = load("preprocessor.joblib")
    model_loaded = load("heart_disease_model.joblib")
    print("✅ Mô hình tối ưu đã được lưu!")

# 🚀 **3. Tạo giao diện nhập liệu bằng tiếng Việt**
# Tạo style cho description_width
style = {'description_width': '150px'}

# Tạo các widget với layout và style
tuoi = widgets.IntText(description="Tuổi:",
                           style=style,
                           layout=widgets.Layout(width='30%'))
gioi_tinh = widgets.Dropdown(options=[("Nữ", 0), ("Nam", 1)], description="Giới tính:",
                                 style=style,
                                 layout=widgets.Layout(width='30%'))
dau_nguc = widgets.Dropdown(options=[("Đau thắt ngực điển hình", 0), ("Đau thắt ngực không điển hình", 1), ("Đau không phải thắt ngực", 2), ("Không triệu chứng", 3)], description="Loại đau ngực:",
                                style=style,
                                layout=widgets.Layout(width='30%'))
huyet_ap = widgets.FloatText(description="Huyết áp nghỉ (mmHg):",
                                 style=style,
                                 layout=widgets.Layout(width='30%'))
cholesterol = widgets.FloatText(description="Cholesterol (mg/dl):",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))
duong_huyet = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đường huyết cao:",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))
dien_tam_do = widgets.Dropdown(options=[("Bình thường", 0), ("Bất thường ST-T", 1), ("Phì đại thất trái", 2)], description="Điện tâm đồ nghỉ:",
                                  style=style,
                                  layout=widgets.Layout(width='30%'))
nhip_tim = widgets.IntText(description="Nhịp tim tối đa:",
                              style=style,
                              layout=widgets.Layout(width='30%'))
dau_nguc_gang_suc = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đau ngực khi gắng sức:",
                                       style=style,
                                       layout=widgets.Layout(width='30%'))
so_mach = widgets.IntSlider(min=0, max=4, description="Số mạch bị hẹp:",
                               style=style,
                               layout=widgets.Layout(width='30%'))
thalassemia = widgets.Dropdown(options=[("Không xác định", 1), ("Bình thường", 3), ("Khiếm khuyết cố định", 6), ("Khiếm khuyết đảo ngược", 7)], description="Thalassemia:",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))

display(tuoi, gioi_tinh, dau_nguc, huyet_ap, cholesterol, duong_huyet, dien_tam_do, nhip_tim, dau_nguc_gang_suc, so_mach, thalassemia)

# 🔥 Nút bấm dự đoán
button = widgets.Button(description="Dự đoán")
output = widgets.Output()

def du_doan_benh_tim(b):
    with output:
        output.clear_output()
        try:
            input_df = pd.DataFrame({
                'age': [tuoi.value],
                'sex': [gioi_tinh.value],
                'cp': [dau_nguc.value],
                'trestbps': [huyet_ap.value],
                'chol': [cholesterol.value],
                'fbs': [duong_huyet.value],
                'restecg': [dien_tam_do.value],
                'thalach': [nhip_tim.value],
                'exang': [dau_nguc_gang_suc.value],
                'ca': [so_mach.value],
                'thal': [thalassemia.value]
            })
            print("Dữ liệu đầu vào để dự đoán:")
            print(input_df.iloc[0])
            input_data = preprocessor_loaded.transform(input_df)
            prediction = model_loaded.predict(input_data)

            # Thêm ngưỡng thủ công cho trường hợp cực đoan



            print("Kết quả dự đoán:", "Có nguy cơ" if prediction[0] == 1 else "Không nguy cơ")
            if prediction[0] == 1:
                print("🔴 Có nguy cơ mắc bệnh tim!")
            else:
                print("🟢 Không có nguy cơ mắc bệnh tim.")
        except NameError:
            print("⚠️ Vui lòng tải file CSV và huấn luyện mô hình trước!")

button.on_click(du_doan_benh_tim)
display(button, output)

FileUpload(value={}, accept='.csv', description='Tải file CSV')

Output()

IntText(value=0, description='Tuổi:', layout=Layout(width='30%'), style=DescriptionStyle(description_width='15…

Dropdown(description='Giới tính:', layout=Layout(width='30%'), options=(('Nữ', 0), ('Nam', 1)), style=Descript…

Dropdown(description='Loại đau ngực:', layout=Layout(width='30%'), options=(('Đau thắt ngực điển hình', 0), ('…

...

Dropdown(description='Đau ngực khi gắng sức:', layout=Layout(width='30%'), options=(('Không', 0), ('Có', 1)), …

IntSlider(value=0, description='Số mạch bị hẹp:', layout=Layout(width='30%'), max=4, style=SliderStyle(descrip…

Dropdown(description='Thalassemia:', layout=Layout(width='30%'), options=(('Không xác định', 1), ('Bình thường…

Button(description='Dự đoán', style=ButtonStyle())

Output()

# Tối ưu mô hình máy học dự đoán bệnh tim

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import ipywidgets as widgets
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from IPython.display import display
import io

# 🚀 **1. Tạo widget để tải file CSV**
upload_button = widgets.FileUpload(accept='.csv', multiple=False, description="Tải file CSV")
upload_output = widgets.Output()

categorical_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'thal']
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'ca']
target_column = 'target'

def on_file_upload(change):
    with upload_output:
        upload_output.clear_output()
        if upload_button.value:
            uploaded_file = list(upload_button.value.values())[0]
            content = uploaded_file['content']
            global df, X, y, X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded
            df = pd.read_csv(io.BytesIO(content))
            print("✅ File CSV đã được tải lên thành công!")
            print("Số hàng trong dữ liệu:", len(df))
            print("Cột trong dữ liệu:", df.columns.tolist())
            print("Dữ liệu mẫu (hàng đầu tiên):")
            print(df.iloc[0])

            X = df[categorical_features + numerical_features]
            y = df[target_column]

            print("5 mẫu có nguy cơ mắc bệnh tim từ dữ liệu gốc:")
            print(df[df[target_column] == 1][categorical_features + numerical_features].head())

            print("Phạm vi giá trị tối đa của các đặc trưng trong các mẫu target=1:")
            for feature in numerical_features:
                print(f"{feature}: max={df[df[target_column] == 1][feature].max()}")

            process_data()

upload_button.observe(on_file_upload, names='value')
display(upload_button, upload_output)

# 🚀 **2. Hàm xử lý dữ liệu và huấn luyện mô hình**
def process_data():
    global X_train, X_test, y_train, y_test, preprocessor_loaded, model_loaded
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    preprocessor = ColumnTransformer([
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    model = RandomForestClassifier(
        n_estimators=100,     # tăng số cây để có thể học nhiều hơn
        max_depth=50,         # cho phép cây sâu hơn nếu cần thiết
        min_samples_leaf=3,   # giảm số mẫu tối thiểu để mỗi lá có nhiều thông tin hơn
        max_features='sqrt',  # bạn có thể thử thay đổi thành 'log2' để xem hiệu quả
        class_weight='balanced',
        random_state=0
    )


    model.fit(X_train_transformed, y_train)
    y_pred = model.predict(X_test_transformed)
    print("🎯 Độ chính xác trên tập kiểm tra:", accuracy_score(y_test, y_pred))
    print("📜 Báo cáo phân loại:\n", classification_report(y_test, y_pred))

    print("Dự đoán trên tập kiểm tra (so sánh với thực tế):")
    for i, (pred, actual) in enumerate(zip(y_pred, y_test)):
        print(f"Mẫu {i}: Dự đoán = {pred}, Thực tế = {actual}")

    feature_importance = pd.DataFrame({
        'Feature': preprocessor.get_feature_names_out(),
        'Importance': model.feature_importances_
    })
    print("Độ quan trọng của các đặc trưng:")
    print(feature_importance.sort_values(by='Importance', ascending=False))

    low_importance = feature_importance[feature_importance['Importance'] < 0.01]['Feature'].tolist()
    print("Loại bỏ:", low_importance)

    dump(preprocessor, "preprocessor.joblib")
    dump(model, "heart_disease_model.joblib")
    print("✅ Mô hình và bộ tiền xử lý đã được lưu!")
    preprocessor_loaded = load("preprocessor.joblib")
    model_loaded = load("heart_disease_model.joblib")

# 🚀 **3. Tạo giao diện nhập liệu bằng tiếng Việt**
# Tạo style cho description_width
style = {'description_width': '150px'}

# Tạo các widget với layout và style
tuoi = widgets.IntText(description="Tuổi:",
                           style=style,
                           layout=widgets.Layout(width='30%'))
gioi_tinh = widgets.Dropdown(options=[("Nữ", 0), ("Nam", 1)], description="Giới tính:",
                                 style=style,
                                 layout=widgets.Layout(width='30%'))
dau_nguc = widgets.Dropdown(options=[("Đau thắt ngực điển hình", 0), ("Đau thắt ngực không điển hình", 1), ("Đau không phải thắt ngực", 2), ("Không triệu chứng", 3)], description="Loại đau ngực:",
                                style=style,
                                layout=widgets.Layout(width='30%'))
huyet_ap = widgets.FloatText(description="Huyết áp nghỉ (mmHg):",
                                 style=style,
                                 layout=widgets.Layout(width='30%'))
cholesterol = widgets.FloatText(description="Cholesterol (mg/dl):",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))
duong_huyet = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đường huyết cao:",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))
dien_tam_do = widgets.Dropdown(options=[("Bình thường", 0), ("Bất thường ST-T", 1), ("Phì đại thất trái", 2)], description="Điện tâm đồ nghỉ:",
                                  style=style,
                                  layout=widgets.Layout(width='30%'))
nhip_tim = widgets.IntText(description="Nhịp tim tối đa:",
                              style=style,
                              layout=widgets.Layout(width='30%'))
dau_nguc_gang_suc = widgets.Dropdown(options=[("Không", 0), ("Có", 1)], description="Đau ngực khi gắng sức:",
                                       style=style,
                                       layout=widgets.Layout(width='30%'))
so_mach = widgets.IntSlider(min=0, max=4, description="Số mạch bị hẹp:",
                               style=style,
                               layout=widgets.Layout(width='30%'))
thalassemia = widgets.Dropdown(options=[("Không xác định", 1), ("Bình thường", 3), ("Khiếm khuyết cố định", 6), ("Khiếm khuyết đảo ngược", 7)], description="Thalassemia:",
                                   style=style,
                                   layout=widgets.Layout(width='30%'))

display(tuoi, gioi_tinh, dau_nguc, huyet_ap, cholesterol, duong_huyet, dien_tam_do, nhip_tim, dau_nguc_gang_suc, so_mach, thalassemia)

# 🔥 Nút bấm dự đoán
button = widgets.Button(description="Dự đoán")
output = widgets.Output()

def du_doan_benh_tim(b):
    with output:
        output.clear_output()
        try:
            input_df = pd.DataFrame({
                'age': [tuoi.value],
                'sex': [gioi_tinh.value],
                'cp': [dau_nguc.value],
                'trestbps': [huyet_ap.value],
                'chol': [cholesterol.value],
                'fbs': [duong_huyet.value],
                'restecg': [dien_tam_do.value],
                'thalach': [nhip_tim.value],
                'exang': [dau_nguc_gang_suc.value],
                'ca': [so_mach.value],
                'thal': [thalassemia.value]
            })
            print("Dữ liệu đầu vào để dự đoán:")
            print(input_df.iloc[0])
            input_data = preprocessor_loaded.transform(input_df)
            prediction = model_loaded.predict(input_data)

            # Thêm ngưỡng thủ công cho trường hợp cực đoan



            print("Kết quả dự đoán:", "Có nguy cơ" if prediction[0] == 1 else "Không nguy cơ")
            if prediction[0] == 1:
                print("🔴 Có nguy cơ mắc bệnh tim!")
            else:
                print("🟢 Không có nguy cơ mắc bệnh tim.")
        except NameError:
            print("⚠️ Vui lòng tải file CSV và huấn luyện mô hình trước!")

button.on_click(du_doan_benh_tim)
display(button, output)

FileUpload(value={}, accept='.csv', description='Tải file CSV')

Output()

IntText(value=0, description='Tuổi:', layout=Layout(width='30%'), style=DescriptionStyle(description_width='15…

Dropdown(description='Giới tính:', layout=Layout(width='30%'), options=(('Nữ', 0), ('Nam', 1)), style=Descript…

Dropdown(description='Loại đau ngực:', layout=Layout(width='30%'), options=(('Đau thắt ngực điển hình', 0), ('…

...

Dropdown(description='Đau ngực khi gắng sức:', layout=Layout(width='30%'), options=(('Không', 0), ('Có', 1)), …

IntSlider(value=0, description='Số mạch bị hẹp:', layout=Layout(width='30%'), max=4, style=SliderStyle(descrip…

Dropdown(description='Thalassemia:', layout=Layout(width='30%'), options=(('Không xác định', 1), ('Bình thường…

Button(description='Dự đoán', style=ButtonStyle())

Output()